# 🌿 Interactive Soil Water Retention Curve Explorer

This notebook uses **ipywidgets** to let you interactively visualise and modify
Mualem–van Genuchten (MVG) hydraulic parameters for peat soils.

**Sections:**
1. **PTF Explorer** — Adjust BD (and optionally depth/OM/peat type) to see how the PTF-predicted SWRC and K(h) change in real time.
2. **Manual Parameter Editor** — Directly drag sliders for θs, α, n, Ks, τ, θr and see instant curve updates.
3. **Range-Constrained Explorer** — Use a fraction slider to sweep through the allowable parameter envelope at a given BD.
4. **Side-by-Side Comparison** — Compare PTF prediction vs. range-constrained (low / mid / high) curves.

In [1]:
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

import math
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

from peat_ptf import (
    MVGParameters,
    ptf_all_data, ptf_all_types_high_bd,
    vg_water_retention, vg_hydraulic_conductivity,
)
from range_peat_ptf import (
    interpolate_ranges, constrain_parameters,
    parameter_at_fraction, parameter_at_fraction_inv,
    named_parameter_sets, get_range_summary,
)

# Interactive matplotlib backend (install ipympl if missing: pip install ipympl)
try:
    %matplotlib widget
except Exception:
    %matplotlib inline
    print('ipympl not available — using inline backend (sliders will redraw, not update in-place)')

print('All imports OK')

All imports OK


---
## 1. PTF Explorer

Adjust **bulk density**, **depth**, **OM**, and **peat type** to see the PTF-predicted
SWRC (left) and K(h) (right) update in real time.

> For BD > 0.2 the type-specific PTFs are not available — the dispatcher
> automatically uses `ptf_all_types_high_bd`.

In [ ]:
# Pressure-head array (shared by all plots)
_H = np.logspace(-2, 5, 600)

def _plot_swrc_k(ax_s, ax_k, params, label, color=None):
    """Draw SWRC and K(h) on the given axes."""
    theta = vg_water_retention(-_H, params)
    ax_s.plot(_H, theta, lw=2, label=label, color=color)
    if not math.isnan(params.Ks):
        K = vg_hydraulic_conductivity(-_H, params)
        ax_k.plot(_H, K, lw=2, label=label, color=color)

def _format_axes(ax_s, ax_k):
    ax_s.set_xscale('log'); ax_s.set_xlabel('|h| (cm)')
    ax_s.set_ylabel(r'$\theta$ (cm³ cm⁻³)'); ax_s.set_title('Water Retention Curve')
    ax_s.set_ylim(0, 1); ax_s.grid(True, alpha=0.3); ax_s.legend(fontsize=8)
    ax_k.set_xscale('log'); ax_k.set_yscale('log'); ax_k.set_xlabel('|h| (cm)')
    ax_k.set_ylabel('K (cm h⁻¹)'); ax_k.set_title('Hydraulic Conductivity')
    ax_k.grid(True, alpha=0.3); ax_k.legend(fontsize=8)

# --- widgets ---
w_bd    = widgets.FloatSlider(value=0.3, min=0.02, max=0.76, step=0.01, description='BD (g/cm³)', style={'description_width': 'initial'}, layout=widgets.Layout(width='420px'))
w_depth = widgets.FloatSlider(value=10,  min=0,    max=200,  step=1,    description='Depth (cm)',  style={'description_width': 'initial'}, layout=widgets.Layout(width='420px'))
w_om    = widgets.FloatSlider(value=90,  min=20,   max=100,  step=1,    description='OM (wt%)',    style={'description_width': 'initial'}, layout=widgets.Layout(width='420px'))
w_type  = widgets.Dropdown(options=['(none)', 'sphagnum', 'woody', 'sedge'], value='(none)', description='Peat type', style={'description_width': 'initial'})

out_ptf = widgets.Output()

def _update_ptf(*_):
    ptype = None if w_type.value == '(none)' else w_type.value
    try:
        params = ptf_all_data(w_bd.value, depth=w_depth.value, OM=w_om.value, peat_type=ptype)
    except Exception as e:
        with out_ptf:
            clear_output(wait=True)
            print(f'⚠️  {e}')
        return
    with out_ptf:
        clear_output(wait=True)
        fig, (ax_s, ax_k) = plt.subplots(1, 2, figsize=(11, 4))
        _plot_swrc_k(ax_s, ax_k, params, f'PTF (BD={w_bd.value:.2f})')
        _format_axes(ax_s, ax_k)
        # Parameter annotation
        info = (f'θs={params.theta_s:.3f}  α={params.alpha:.4f}\n'
                f'n={params.n:.4f}  Ks={params.Ks:.4f}\n'
                f'τ={params.tau:.2f}  m={params.m:.4f}')
        ax_s.text(0.97, 0.97, info, transform=ax_s.transAxes, fontsize=7,
                  va='top', ha='right', fontfamily='monospace',
                  bbox=dict(boxstyle='round,pad=0.3', fc='wheat', alpha=0.6))
        fig.tight_layout()
        plt.show()

for w in [w_bd, w_depth, w_om, w_type]:
    w.observe(_update_ptf, names='value')

ui_ptf = widgets.VBox([
    widgets.HBox([w_bd, w_type]),
    widgets.HBox([w_depth, w_om]),
    out_ptf,
])
display(ui_ptf)
_update_ptf()  # initial draw

---
## 2. Manual Parameter Editor

Drag the sliders to set **any** MVG parameter values directly and see the
curves update immediately.  Use this when you want to hand-tune values for
your simulation software.

> The α and Ks sliders are **logarithmic** so you can span several orders
> of magnitude.

In [3]:
# --- manual parameter sliders ---
m_ts  = widgets.FloatSlider(value=0.85, min=0.3, max=1.0,  step=0.005, description='θs',   style={'description_width': '40px'}, layout=widgets.Layout(width='400px'))
m_tr  = widgets.FloatSlider(value=0.0,  min=0.0, max=0.3,  step=0.005, description='θr',   style={'description_width': '40px'}, layout=widgets.Layout(width='400px'))
m_la  = widgets.FloatLogSlider(value=0.02, min=-3, max=2,  step=0.01,  description='α',    style={'description_width': '40px'}, layout=widgets.Layout(width='400px'))
m_n   = widgets.FloatSlider(value=1.2,  min=1.01, max=3.0, step=0.005, description='n',    style={'description_width': '40px'}, layout=widgets.Layout(width='400px'))
m_ks  = widgets.FloatLogSlider(value=1.0, min=-3, max=4,   step=0.01,  description='Ks',   style={'description_width': '40px'}, layout=widgets.Layout(width='400px'))
m_tau = widgets.FloatSlider(value=0.5,  min=-6,  max=6,    step=0.1,   description='τ',    style={'description_width': '40px'}, layout=widgets.Layout(width='400px'))

out_manual = widgets.Output()

def _update_manual(*_):
    params = MVGParameters(
        theta_s=m_ts.value, alpha=m_la.value, n=m_n.value,
        Ks=m_ks.value, tau=m_tau.value, theta_r=m_tr.value,
    )
    with out_manual:
        clear_output(wait=True)
        fig, (ax_s, ax_k) = plt.subplots(1, 2, figsize=(11, 4))
        _plot_swrc_k(ax_s, ax_k, params, 'Manual', color='#2b8cbe')
        _format_axes(ax_s, ax_k)
        info = (f'θs={params.theta_s:.3f}  θr={params.theta_r:.3f}\n'
                f'α={params.alpha:.4e}  n={params.n:.4f}\n'
                f'Ks={params.Ks:.4e}  τ={params.tau:.2f}')
        ax_s.text(0.97, 0.97, info, transform=ax_s.transAxes, fontsize=7,
                  va='top', ha='right', fontfamily='monospace',
                  bbox=dict(boxstyle='round,pad=0.3', fc='lightyellow', alpha=0.7))
        fig.tight_layout()
        plt.show()

for w in [m_ts, m_tr, m_la, m_n, m_ks, m_tau]:
    w.observe(_update_manual, names='value')

ui_manual = widgets.VBox([
    widgets.HBox([m_ts, m_tr]),
    widgets.HBox([m_la, m_n]),
    widgets.HBox([m_ks, m_tau]),
    out_manual,
])
display(ui_manual)
_update_manual()

---
## 3. Range-Constrained Explorer

Slide **BD** and a **fraction** (0 = lower bound, 1 = upper bound) to sweep
through the allowable parameter envelope from `range_peat_ptf`.

The **grey band** shows the full SWRC envelope (low → high).  The **blue curve**
is the set at the current fraction.  The **dashed orange curve** is the raw PTF
prediction (clamped if outside range).

This is the key tool for finding a parameter set that your simulation software
can handle while staying physically plausible.

In [4]:
parameter_at_fraction(0.5, 1)

MVGParameters(θs=0.7315, θr=0.0000, α=0.1122, n=1.3646, Ks=5.0119, τ=0.5000)

In [5]:
r_bd   = widgets.FloatSlider(value=0.4, min=0.21, max=0.76, step=0.01, description='BD (g/cm³)',  style={'description_width': 'initial'}, layout=widgets.Layout(width='420px'))
r_frac = widgets.FloatSlider(value=0.5, min=0.0,  max=1.0,  step=0.01, description='Fraction',    style={'description_width': 'initial'}, layout=widgets.Layout(width='420px'))

out_range = widgets.Output()

def _update_range(*_):
    bd = r_bd.value
    frac = r_frac.value

    p_lo   = parameter_at_fraction(bd, 0.0)
    p_hi   = parameter_at_fraction(bd, 1.0)
    p_frac = parameter_at_fraction(bd, frac)
    p_ptf  = constrain_parameters(bd)

    theta_lo = vg_water_retention(-_H, p_lo)
    theta_hi = vg_water_retention(-_H, p_hi)
    theta_fr = vg_water_retention(-_H, p_frac)
    theta_ptf = vg_water_retention(-_H, p_ptf)

    with out_range:
        clear_output(wait=True)
        fig, (ax_s, ax_k) = plt.subplots(1, 2, figsize=(11, 4.5))

        # Envelope
        ax_s.fill_between(_H, theta_lo, theta_hi, alpha=0.15, color='grey', label='Allowable envelope')
        ax_s.plot(_H, theta_fr, lw=2.5, color='#2b8cbe', label=f'Fraction = {frac:.2f}')
        ax_s.plot(_H, theta_ptf, lw=1.5, ls='--', color='#e66101', label='PTF (clamped)')
        _format_axes(ax_s, ax_k)

        # K(h)
        if not math.isnan(p_lo.Ks):
            K_lo = vg_hydraulic_conductivity(-_H, p_lo)
            K_hi = vg_hydraulic_conductivity(-_H, p_hi)
            ax_k.fill_between(_H, K_lo, K_hi, alpha=0.15, color='grey', label='Envelope')
        K_fr = vg_hydraulic_conductivity(-_H, p_frac)
        K_ptf = vg_hydraulic_conductivity(-_H, p_ptf)
        ax_k.plot(_H, K_fr, lw=2.5, color='#2b8cbe', label=f'Fraction = {frac:.2f}')
        ax_k.plot(_H, K_ptf, lw=1.5, ls='--', color='#e66101', label='PTF (clamped)')
        _format_axes(ax_s, ax_k)

        # Parameter table
        info = (f'--- Fraction {frac:.2f} ---\n'
                f'θs={p_frac.theta_s:.3f}  α={p_frac.alpha:.4e}\n'
                f'n ={p_frac.n:.4f}   Ks={p_frac.Ks:.4e}\n'
                f'τ ={p_frac.tau:.2f}')
        ax_s.text(0.97, 0.97, info, transform=ax_s.transAxes, fontsize=7,
                  va='top', ha='right', fontfamily='monospace',
                  bbox=dict(boxstyle='round,pad=0.3', fc='lightcyan', alpha=0.7))

        # Range summary text
        rs = get_range_summary(bd)
        rtxt = (f'--- Ranges at BD={bd:.2f} ---\n'
                f'Ks:  [{rs["Ks"]["min"]:.3e}, {rs["Ks"]["max"]:.3e}]\n'
                f'α:   [{rs["alpha"]["min"]:.3e}, {rs["alpha"]["max"]:.3e}]\n'
                f'n:   [{rs["n"]["min"]:.4f}, {rs["n"]["max"]:.4f}]')
        ax_k.text(0.97, 0.97, rtxt, transform=ax_k.transAxes, fontsize=7,
                  va='top', ha='right', fontfamily='monospace',
                  bbox=dict(boxstyle='round,pad=0.3', fc='lightyellow', alpha=0.7))

        fig.suptitle(f'Range-Constrained Explorer  (BD = {bd:.2f} g cm⁻³)', fontsize=11, y=1.02)
        fig.tight_layout()
        plt.show()

for w in [r_bd, r_frac]:
    w.observe(_update_range, names='value')

ui_range = widgets.VBox([widgets.HBox([r_bd, r_frac]), out_range])
display(ui_range)
_update_range()

---
## 3b. Inverted α–n Explorer

Same idea as Section 3, but here **α and n move in opposite directions**:

| fraction → | 0 (low) | 1 (high) |
|---|---|---|
| **Ks** | low | high |
| **n** | low | **high** |
| **α** | **high** | low |

This produces the physically common pairing of a **steep curve** (high n)
with a **small air-entry** (low α), and vice versa.  Compare the blue
(inverted) curve against the dashed orange (normal fraction) to see the
difference.

In [ ]:
i_bd   = widgets.FloatSlider(value=0.4, min=0.21, max=0.76, step=0.01, description='BD (g/cm³)',  style={'description_width': 'initial'}, layout=widgets.Layout(width='420px'))
i_frac = widgets.FloatSlider(value=0.5, min=0.0,  max=1.0,  step=0.01, description='Fraction',    style={'description_width': 'initial'}, layout=widgets.Layout(width='420px'))

out_inv = widgets.Output()

def _update_inv(*_):
    bd = i_bd.value
    frac = i_frac.value

    p_inv    = parameter_at_fraction_inv(bd, frac)
    p_normal = parameter_at_fraction(bd, frac)
    p_ptf    = constrain_parameters(bd)

    # Envelope (same as section 3)
    p_lo = parameter_at_fraction(bd, 0.0)
    p_hi = parameter_at_fraction(bd, 1.0)
    theta_lo = vg_water_retention(-_H, p_lo)
    theta_hi = vg_water_retention(-_H, p_hi)

    with out_inv:
        clear_output(wait=True)
        fig, (ax_s, ax_k) = plt.subplots(1, 2, figsize=(11, 4.5))

        # Envelope
        ax_s.fill_between(_H, theta_lo, theta_hi, alpha=0.12, color='grey', label='Envelope')

        # Inverted curve (main)
        theta_inv = vg_water_retention(-_H, p_inv)
        ax_s.plot(_H, theta_inv, lw=2.5, color='#2b8cbe', label=f'Inverted (f={frac:.2f})')

        # Normal fraction (reference)
        theta_norm = vg_water_retention(-_H, p_normal)
        ax_s.plot(_H, theta_norm, lw=1.5, ls='--', color='#e66101', label=f'Normal (f={frac:.2f})')

        # PTF
        theta_ptf = vg_water_retention(-_H, p_ptf)
        ax_s.plot(_H, theta_ptf, lw=1, ls=':', color='black', label='PTF (clamped)')
        _format_axes(ax_s, ax_k)

        # K(h)
        if not math.isnan(p_lo.Ks):
            K_lo = vg_hydraulic_conductivity(-_H, p_lo)
            K_hi = vg_hydraulic_conductivity(-_H, p_hi)
            ax_k.fill_between(_H, K_lo, K_hi, alpha=0.12, color='grey', label='Envelope')
        K_inv  = vg_hydraulic_conductivity(-_H, p_inv)
        K_norm = vg_hydraulic_conductivity(-_H, p_normal)
        K_ptf  = vg_hydraulic_conductivity(-_H, p_ptf)
        ax_k.plot(_H, K_inv,  lw=2.5, color='#2b8cbe',  label=f'Inverted (f={frac:.2f})')
        ax_k.plot(_H, K_norm, lw=1.5, ls='--', color='#e66101', label=f'Normal (f={frac:.2f})')
        ax_k.plot(_H, K_ptf,  lw=1,   ls=':',  color='black',   label='PTF (clamped)')
        _format_axes(ax_s, ax_k)

        # Annotations
        inv_info = (f'--- Inverted f={frac:.2f} ---\n'
                    f'α={p_inv.alpha:.4e}  (← high→low)\n'
                    f'n={p_inv.n:.4f}       (← low→high)\n'
                    f'Ks={p_inv.Ks:.4e}')
        norm_info = (f'--- Normal f={frac:.2f} ---\n'
                     f'α={p_normal.alpha:.4e}\n'
                     f'n={p_normal.n:.4f}\n'
                     f'Ks={p_normal.Ks:.4e}')
        ax_s.text(0.97, 0.97, inv_info, transform=ax_s.transAxes, fontsize=7,
                  va='top', ha='right', fontfamily='monospace',
                  bbox=dict(boxstyle='round,pad=0.3', fc='lightcyan', alpha=0.7))
        ax_s.text(0.97, 0.68, norm_info, transform=ax_s.transAxes, fontsize=7,
                  va='top', ha='right', fontfamily='monospace',
                  bbox=dict(boxstyle='round,pad=0.3', fc='bisque', alpha=0.7))

        fig.suptitle(f'Inverted α–n Explorer  (BD = {bd:.2f} g cm⁻³)', fontsize=11, y=1.02)
        fig.tight_layout()
        plt.show()

for w in [i_bd, i_frac]:
    w.observe(_update_inv, names='value')

ui_inv = widgets.VBox([widgets.HBox([i_bd, i_frac]), out_inv])
display(ui_inv)
_update_inv()

---
## 4. Side-by-Side Comparison: PTF vs Low / Mid / High

Overlay the three named sets (`low`, `mid`, `high`) together with the clamped
PTF prediction so you can directly compare them.

In [7]:
c_bd = widgets.FloatSlider(value=0.4, min=0.21, max=0.76, step=0.01, description='BD (g/cm³)', style={'description_width': 'initial'}, layout=widgets.Layout(width='420px'))

out_cmp = widgets.Output()

def _update_cmp(*_):
    bd = c_bd.value
    named = named_parameter_sets(bd)
    p_ptf = constrain_parameters(bd)

    colors = {'low': '#1b9e77', 'mid': '#7570b3', 'high': '#d95f02'}

    with out_cmp:
        clear_output(wait=True)
        fig, (ax_s, ax_k) = plt.subplots(1, 2, figsize=(11, 4.5))

        for lbl, p in named.items():
            _plot_swrc_k(ax_s, ax_k, p, lbl.capitalize(), color=colors[lbl])

        # PTF (clamped)
        _plot_swrc_k(ax_s, ax_k, p_ptf, 'PTF (clamped)', color='black')
        ax_s.lines[-1].set_linestyle('--')
        if ax_k.lines:
            ax_k.lines[-1].set_linestyle('--')

        _format_axes(ax_s, ax_k)
        fig.suptitle(f'Comparison at BD = {bd:.2f} g cm⁻³', fontsize=11, y=1.02)
        fig.tight_layout()
        plt.show()

c_bd.observe(_update_cmp, names='value')
display(widgets.VBox([c_bd, out_cmp]))
_update_cmp()

---
## 5. Export Selected Parameters

Run the cell below after choosing your preferred fraction / BD above.
It prints the parameters in a format you can copy-paste into your
simulation input files.

In [8]:
# --- Change these to your chosen values ---
export_BD   = 0.4
export_frac = 0.3   # 0=low, 0.5=mid, 1=high

p = parameter_at_fraction(export_BD, export_frac)

print(f'BD = {export_BD} g/cm³   |   Fraction = {export_frac}')
print('=' * 48)
print(f'theta_s  = {p.theta_s:.6f}   # cm³ cm⁻³')
print(f'theta_r  = {p.theta_r:.6f}   # cm³ cm⁻³')
print(f'alpha    = {p.alpha:.6e}   # cm⁻¹')
print(f'n        = {p.n:.6f}')
print(f'm        = {p.m:.6f}   # (1 - 1/n)')
print(f'Ks       = {p.Ks:.6e}   # cm h⁻¹')
print(f'tau      = {p.tau:.6f}')

BD = 0.4 g/cm³   |   Fraction = 0.3
theta_s  = 0.775200   # cm³ cm⁻³
theta_r  = 0.000000   # cm³ cm⁻³
alpha    = 8.912509e-03   # cm⁻¹
n        = 1.145513
m        = 0.127029   # (1 - 1/n)
Ks       = 7.413102e-02   # cm h⁻¹
tau      = 0.500000
